<a href="https://colab.research.google.com/github/graemegalloway/Carbon-Footprint-Calculator/blob/main/Empirical_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required package if not already installed
!pip install pyreadstat

import pandas as pd
import pyreadstat

# Define file paths (assuming files are in Google Colab's '/content/' directory)
file_paths = {
    "2017": "/content/lfs-71M0001-E-2017-march_F1.sav",
    "2019": "/content/lfs-71M0001-E-2019-march_F1.sav",
    "2024": "/content/LFS_March_2024.sav"
}

# Load data and add a 'Year' column to each dataframe
dfs = []
for year, path in file_paths.items():
    df, meta = pyreadstat.read_sav(path)  # Read SPSS file
    df["Year"] = int(year)  # Add a year column
    dfs.append(df)

# Concatenate all dataframes into one
df_combined = pd.concat(dfs, ignore_index=True)

# Save merged data as a CSV (optional, for further use)
df_combined.to_csv("/content/Merged_LFS_Data.csv", index=False)

# Display the first few rows to confirm merging worked correctly
df_combined.head()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.5 MB/s eta 0:00:00


,REC_NUM,SURVYEAR,SURVMNTH,LFSSTAT,PROV,CMA,AGE_12,AGE_6,SEX,MARSTAT,...,LKOTHERN,PRIORACT,YNOLOOK,TLOLOOK,SCHOOLN,EFAMTYPE,AGYOWNK,FINALWT,Year,NOC_43
0,1.0,2017.0,3.0,1.0,35.0,5.0,9.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,4.0,4.0,380.0,2017,NaN
1,2.0,2017.0,3.0,1.0,59.0,9.0,6.0,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,1.0,9.0,3.0,720.0,2017,NaN
2,3.0,2017.0,3.0,1.0,59.0,0.0,1.0,2.0,2.0,6.0,...,NaN,NaN,NaN,NaN,1.0,3.0,NaN,85.0,2017,NaN
3,4.0,2017.0,3.0,4.0,35.0,0.0,10.0,NaN,1.0,3.0,...,NaN,NaN,NaN,NaN,1.0,18.0,NaN,245.0,2017,NaN
4,5.0,2017.0,3.0,4.0,24.0,0.0,1.0,2.0,1.0,6.0,...,NaN,NaN,NaN,NaN,2.0,3.0,NaN,67.0,2017,NaN


In [6]:
# Total number of observations in the uncleaned master dataset
total_observations = df_combined.shape[0]
print(f"Total Observations in the Master File: {total_observations}")



Total Observations in the Master File: 314676


In [7]:
# Total number of observations in the uncleaned master dataset per year
observations_per_year = df_combined["Year"].value_counts().sort_index()

# Display the counts
print("Number of Observations per Year:")
print(observations_per_year)


Number of Observations per Year:
Year
2017    103030
2019    100364
2024    111282
Name: count, dtype: int64


In [18]:
# filtered dataset number of observations (Q1 part two)
filtered_df = df_combined[
    (df_combined["AGE_12"] >= 3) & (df_combined["AGE_12"] <= 10) &  # Age 25-64
    (df_combined["HRLYEARN"] > 0) &  # Positive hourly wage
    (df_combined["COWMAIN"].isin([1, 2])) &  # Public or private sector workers
    (df_combined["SCHOOLN"] == 1)  # Non-students
]

# Total number of observations after filtering
filtered_total = filtered_df.shape[0]
print(f"Total Observations After Filtering: {filtered_total}")

# Number of observations per year after filtering
filtered_per_year = filtered_df["Year"].value_counts().sort_index()
print("Observations Per Year After Filtering:")
print(filtered_per_year)

# Convert to a DataFrame for better visualization
obs_filtered_summary = pd.DataFrame({"Year": filtered_per_year.index, "Filtered Observations": filtered_per_year.values})
print(obs_filtered_summary)


Total Observations After Filtering: 126587
Observations Per Year After Filtering:
Year
2017    41235
2019    40554
2024    44798
Name: count, dtype: int64
   Year  Filtered Observations
0  2017                  41235
1  2019                  40554
2  2024                  44798


In [20]:
# Create a copy of the filtered dataset to avoid modifying the original (QUESTION Two)
q2_df = filtered_df.copy()

# Generate the "education" variable based on EDUC categories
def categorize_education(educ):
    if 0 <= educ <= 8:  # 0 to 8 years and Some high school
        return 1  # Less than High-school
    elif 9 <= educ <= 10:  # High school graduate and Some postsecondary
        return 2  # High-School
    elif educ == 11:  # Postsecondary certificate or diploma
        return 3  # Community college
    elif educ == 12:  # Bachelor's degree
        return 4  # University
    elif educ >= 13:  # Above bachelor's degree
        return 5  # Post-graduate
    else:
        return None  # Handle unexpected cases

# Apply education category mapping
q2_df.loc[:, "Education"] = q2_df["EDUC"].apply(categorize_education)

# Generate "union" indicator (1 = Unionized, 0 = Non-unionized)
q2_df.loc[:, "Union"] = q2_df["UNION"].apply(lambda x: 1 if x in [1, 2] else 0)

# Generate "public sector" indicator (1 = Public, 0 = Private)
q2_df.loc[:, "Public_Sector"] = q2_df["COWMAIN"].apply(lambda x: 1 if x == 1 else 0)

# Select relevant variables for further analysis
selected_variables = q2_df[["HRLYEARN", "Education", "SEX", "Public_Sector", "Union"]]

# Display the first few rows of the updated dataset
selected_variables.head()


,HRLYEARN,Education,SEX,Public_Sector,Union
1,25.75,1,2.0,0,0
5,34.09,1,2.0,0,0
6,12.00,1,1.0,0,0
10,11.86,1,2.0,0,0
12,20.00,1,1.0,0,0


In [24]:
# Compute mean and standard deviation for continuous variable (the summary statistics for the continuous variables Q2)
summary_continuous = q2_df["HRLYEARN"].agg(["mean", "std"]).to_frame(name="HRLYEARN")

    # Display summary statistics
print("Summary Statistics - Continuous Variables:")
print(summary_continuous)

Summary Statistics - Continuous Variables:
       HRLYEARN
mean  31.376676
std   16.071996


In [26]:
# Define categorical variables (the tabulaions for categorical variables Q2)
categorical_vars = ["Education", "SEX", "Public_Sector", "Union"]

# Compute tabulations for categorical variables and format them
summary_categorical = {}
for var in categorical_vars:
    df_counts = q2_df[var].value_counts().reset_index()
    df_counts.columns = [var, "Count"]
    df_counts = df_counts.sort_values(by=var).reset_index(drop=True)
    summary_categorical[var] = df_counts

# Display categorical variable distributions in a clean format
print("Category Counts for Categorical Variables:\n")
for var_name, df in summary_categorical.items():
    print(f"{var_name} Distribution:\n")
    print(df.to_string(index=False))
    print("\n" + "-"*40 + "\n")  # Separator for clarity



Category Counts for Categorical Variables:

Education Distribution:

 Education  Count
         1 126587

----------------------------------------

SEX Distribution:

 SEX  Count
 1.0  62851
 2.0  63736

----------------------------------------

Public_Sector Distribution:

 Public_Sector  Count
             0  88903
             1  37684

----------------------------------------

Union Distribution:

 Union  Count
     0  81082
     1  45505

----------------------------------------



In [28]:
# Question Three Code (also has question 4 in it as well tehcnilcally)
import numpy as np
import statsmodels.api as sm

# Create a copy of the dataset to avoid modifying q2_df
q3_df = q2_df.copy()

# Transform SEX into a 0/1 Female indicator (Female = 1, Male = 0)
q3_df["Female"] = q3_df["SEX"].apply(lambda x: 1 if x == 2 else 0)

# Compute the natural log of hourly earnings (avoid log(0) by replacing 0 or negative values with NaN)
q3_df["Log_HRLYEARN"] = np.log(q3_df["HRLYEARN"].replace(0, np.nan))

# Drop NaN values from the log transformation
q3_df = q3_df.dropna(subset=["Log_HRLYEARN"])

# Define independent variables (Education, Female, Public Sector)
X = q3_df[["Education", "Female", "Public_Sector"]]
X = sm.add_constant(X)  # Add constant for regression

# Define dependent variable (Log of Hourly Earnings)
y = q3_df["Log_HRLYEARN"]

# Run OLS regression
model = sm.OLS(y, X).fit()

# Display regression results
print("Log Wage Regression Results:\n")
print(model.summary())


Log Wage Regression Results:

                            OLS Regression Results                            
Dep. Variable:           Log_HRLYEARN   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     7531.
Date:                Tue, 04 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:34:59   Log-Likelihood:                -76951.
No. Observations:              126587   AIC:                         1.539e+05
Df Residuals:                  126584   BIC:                         1.539e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Education       

In [29]:
#question five code (regression two)
import statsmodels.api as sm

# Create a copy of q3_df to avoid modifying it directly
q5_df = q3_df.copy()

# Define independent variables for Regression 2 (Education, Female, Public Sector, and Union)
X2 = q5_df[["Education", "Female", "Public_Sector", "Union"]]
X2 = sm.add_constant(X2)  # Add constant for regression

# Define dependent variable (Log of Hourly Earnings)
y2 = q5_df["Log_HRLYEARN"]

# Run OLS regression for Regression 2
model2 = sm.OLS(y2, X2).fit()

# Print regression results
print("Regression 2: Log Wage Regression Results")
print(model2.summary())


Regression 2: Log Wage Regression Results
                            OLS Regression Results                            
Dep. Variable:           Log_HRLYEARN   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     5024.
Date:                Tue, 04 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:36:51   Log-Likelihood:                -76947.
No. Observations:              126587   AIC:                         1.539e+05
Df Residuals:                  126583   BIC:                         1.539e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Educ

In [30]:
#Question six code (regression three)
import statsmodels.api as sm

# Create a copy of q5_df to avoid modifying the original dataset
q6_df = q5_df.copy()

# Create an interaction term for Public Sector and Female
q6_df["Public_Sector_Female"] = q6_df["Public_Sector"] * q6_df["Female"]

# Define independent variables for Regression 3 (including the interaction term)
X3 = q6_df[["Education", "Female", "Public_Sector", "Union", "Public_Sector_Female"]]
X3 = sm.add_constant(X3)  # Add constant for regression

# Define dependent variable (Log of Hourly Earnings)
y3 = q6_df["Log_HRLYEARN"]

# Run OLS regression for Regression 3
model3 = sm.OLS(y3, X3).fit()

# Print regression results
print("Regression 3: Interaction Term Results")
print(model3.summary())


Regression 3: Interaction Term Results
                            OLS Regression Results                            
Dep. Variable:           Log_HRLYEARN   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     3912.
Date:                Tue, 04 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:38:11   Log-Likelihood:                -76689.
No. Observations:              126587   AIC:                         1.534e+05
Df Residuals:                  126582   BIC:                         1.534e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [42]:
#Question Seven (may be wrong)

# Convert boolean columns (dummies) to int (0/1)
X4 = X4.astype(int)

# Run OLS regression again
model4 = sm.OLS(y4, X4).fit()

# Print regression results
print("\nRegression 4: Log Wage Regression with Additional Controls")
print(model4.summary())



Regression 4: Log Wage Regression with Additional Controls
                            OLS Regression Results                            
Dep. Variable:           Log_HRLYEARN   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     2092.
Date:                Tue, 04 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:46:35   Log-Likelihood:                -60713.
No. Observations:              126587   AIC:                         1.215e+05
Df Residuals:                  126559   BIC:                         1.218e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [45]:
#Question Eight (might be wrong)
import statsmodels.api as sm
import pandas as pd

# Create copies of q7_df for separate regressions by gender
q8_men_df = q7_df[q7_df["Female"] == 0].copy()  # Male dataset
q8_women_df = q7_df[q7_df["Female"] == 1].copy()  # Female dataset

# Define independent variables (same as Regression 4)
X_men = q8_men_df[["Education", "Public_Sector", "Union", "AGE_12", "MARSTAT"]]
X_women = q8_women_df[["Education", "Public_Sector", "Union", "AGE_12", "MARSTAT"]]

# Add occupation and year dummies
X_men = pd.concat([X_men, q8_men_df.filter(like="NAICS_21_"), q8_men_df.filter(like="Year_")], axis=1)
X_women = pd.concat([X_women, q8_women_df.filter(like="NAICS_21_"), q8_women_df.filter(like="Year_")], axis=1)

# Convert all dummies to numeric
X_men = X_men.astype(int)
X_women = X_women.astype(int)

# Add constant for regression
X_men = sm.add_constant(X_men)
X_women = sm.add_constant(X_women)

# Define dependent variable (Log of Hourly Earnings)
y_men = q8_men_df["Log_HRLYEARN"]
y_women = q8_women_df["Log_HRLYEARN"]

# Run separate OLS regressions
model_men = sm.OLS(y_men, X_men).fit()
model_women = sm.OLS(y_women, X_women).fit()

# Print regression results
print("\nRegression 5: Men - Log Wage Regression")
print(model_men.summary())

print("\nRegression 6: Women - Log Wage Regression")
print(model_women.summary())



Regression 5: Men - Log Wage Regression
                            OLS Regression Results                            
Dep. Variable:           Log_HRLYEARN   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.277
Method:                 Least Squares   F-statistic:                     928.2
Date:                Tue, 04 Mar 2025   Prob (F-statistic):               0.00
Time:                        01:48:15   Log-Likelihood:                -31458.
No. Observations:               62851   AIC:                         6.297e+04
Df Residuals:                   62824   BIC:                         6.321e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Educa